In [1]:
import torch
import torch.nn as nn

import random

import pandas as pd
import numpy as np

import os
import sys

import hydroeval as he

import math

import time

import matplotlib.pyplot as plt

from ...SWE_Prediction_Attention.DataLoader.TDL import TemporalDataLoader

In [2]:
snotel_locations_info = pd.read_csv('./Data/Snotel_Locations_Filtered_v3.csv')

snotel_locations_info['Southness'] = [ math.sin(value['Slope_tif1_x']) * math.cos(value['Aspect_tif_x']) for index,value in snotel_locations_info.iterrows()]


In [3]:
tp_dataLoader = TemporalDataLoader()

tp_dataLoader.cleanDataJunk()
tp_dataLoader.prepareInputsAndOutputs()

tp_training_input , tp_training_output = tp_dataLoader.getTrainingData()
tp_testing_data_1,tp_testing_data_2, tp_testing_data_3 , tp_testing_data_4,tp_testing_data_5 = tp_dataLoader.getTestingData()


tp_testing_input_1 , tp_testing_output_1 = tp_testing_data_1
tp_testing_input_2 , tp_testing_output_2 = tp_testing_data_2
tp_testing_input_3 , tp_testing_output_3 = tp_testing_data_3
tp_testing_input_4 , tp_testing_output_4 = tp_testing_data_4
tp_testing_input_5 , tp_testing_output_5 = tp_testing_data_5

In [4]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        batch_size, seq_len , embedding_dim = x.shape
        
        x = x + self.pe[:x.size(0)]
        x = self.dropout(x)
        
        seq_len , batch_size, embedding_dim = x.shape
        
        return x.reshape(batch_size,seq_len,embedding_dim)

    
class SWETransformer(nn.Module):
    def __init__(self, encoder_input_dim = 20, model_dim = 64, n_output_heads = 1,window = 3,
                  seq_length = 10):
        super().__init__()
        
        # Storing the passed argument on the class definition.
        
        self.model_dim = model_dim
        self.encoder_input_dim = encoder_input_dim
        self.n_output_heads = n_output_heads
        self.seq_length = seq_length
        
        # Linear Layers for the input.
        
        self.input_embed_1 = torch.nn.Linear(self.encoder_input_dim , int(self.model_dim/2))
        self.input_embed_2 = torch.nn.Linear(int(self.model_dim/2) , self.model_dim)

        self.input_dropout = torch.nn.Dropout(p = 0.10)
        
        # Positional Encoding
        self.positional_encoding = PositionalEncoding(self.model_dim,0.2)
           
        # Transformer model definition.
        
        self.encoder_layer_1 = nn.TransformerEncoderLayer(d_model=self.model_dim, nhead=16)
        self.transformers_1 = torch.nn.TransformerEncoder(self.encoder_layer_1 , num_layers = 24)
        
        self.encoder_layer_2 = nn.TransformerEncoderLayer(d_model=self.model_dim, nhead=8)
        self.transformers_2 = torch.nn.TransformerEncoder(self.encoder_layer_2, num_layers = 16, norm = None) 
        
        # Dimension Reduction.
        
        initial_dim  = self.model_dim
        
        self.dim_red_1 = torch.nn.Linear(initial_dim * 2 , int(initial_dim/2))
        self.dim_red_2 = torch.nn.Linear(int(initial_dim/2) , int(initial_dim/2))
        self.dim_red_3 = torch.nn.Linear(int(initial_dim/2) , int(initial_dim/4))
        self.dim_red_4 = torch.nn.Linear(int(initial_dim/4) , int(initial_dim/8))
        
        self.dim_red_dropout = torch.nn.Dropout(p = 0.05)
        
        # Final output layer for the model.
        
        self.decoder_layer_1 = torch.nn.Linear(self.seq_length * int(self.model_dim/8) ,self.seq_length * int(self.model_dim/16))    
        self.decoder_layer_2 = torch.nn.Linear(self.seq_length * int(self.model_dim/16), self.seq_length)
        
        # Activation Functions
        
        self.activation_relu = torch.nn.ReLU()
        self.activation_identity = torch.nn.Identity()
        self.activation_gelu = torch.nn.GELU()
        self.activation_tanh = torch.nn.Tanh()
        self.activation_sigmoid = torch.nn.Sigmoid()
        
        # Dropout Functions 
        
        self.dropout_5 = torch.nn.Dropout(p = 0.05)
        self.dropout_10 = torch.nn.Dropout(p = 0.10)
        self.dropout_15 = torch.nn.Dropout(p = 0.15)
        self.dropout_20 = torch.nn.Dropout(p = 0.20)
        
        
    def forward(self,encoder_inputs):
        
        # Converting to the torch array.
        
        encoder_inputs = torch.from_numpy(encoder_inputs).to(dtype= torch.float32)
                
        # Getting the configuration of the passed data.
        
        encoder_batch_size, encoder_sequence_length , encoder_input_dim = encoder_inputs.shape
                
        # Embedding the daily data passed to the model for the locations.
        
        
        embed_input_x = encoder_inputs.reshape(-1,self.encoder_input_dim)
        
        embed_input_x = self.input_embed_1(embed_input_x)
        embed_input_x = self.activation_gelu(embed_input_x)
        
        embed_input_x = self.input_embed_2(embed_input_x)
        embed_input_x = self.activation_gelu(embed_input_x)
        
        embed_input_x = embed_input_x.reshape(encoder_batch_size, encoder_sequence_length, self.model_dim)
        
        # Applying positional encoding.
        
        x = self.positional_encoding(embed_input_x)
        
        # Applying the transformer layer.

        x = self.transformers_1(embed_input_x)
        #x = self.transformers_2(x)
        
        x = x.reshape(-1, self.model_dim)
        embed_input_x = embed_input_x.reshape(-1,self.model_dim)
        
        x = torch.cat((x , embed_input_x),1)
        
        # Dim reduction layer.
        
        x = self.dim_red_1(x) 
        x= self.dropout_20(x)
        
        x = self.dim_red_2(x)
        x = self.activation_relu(x)
        x= self.dropout_20(x)

        x = self.dim_red_3(x)
        x= self.dropout_20(x)
        
        x= self.dim_red_4(x)
        x = self.activation_relu(x)
        x= self.dropout_20(x)
        
        
        
        # Final layer for the output.
        
        x = x.reshape(-1, encoder_sequence_length * int(self.model_dim/8))
        
        x= self.decoder_layer_1(x)
        x= self.activation_gelu(x)
        x = self.dropout_10(x)
        
        x = self.decoder_layer_2(x)
        x = self.activation_identity(x)
        
        x= x.reshape(encoder_batch_size , encoder_sequence_length , self.n_output_heads)
        
        return x
    


In [5]:
swe_model_tp = SWETransformer(encoder_input_dim = 19, model_dim = 512, n_output_heads = 1, seq_length = 270)

In [6]:
# Loss Function 
mean_squared_error_tp = nn.MSELoss()

In [7]:
# Optimizer

optimizer_tp = torch.optim.AdamW(swe_model_tp.parameters(), lr= 0.0001, weight_decay = 0.0001)
scheduler_tp = torch.optim.lr_scheduler.StepLR(optimizer_tp, step_size = 3 ,gamma = 0.6, last_epoch= -1, verbose=False)

In [8]:

def TrainModelTP(train_inputs, train_outputs, epoch_number, data_type= "Temporal"):
    loss_value = 0
    total_loss = 0
    total_batches = 0
    
    for input_index , batch_input in enumerate(train_inputs):
        
        total_batches +=1
        
        optimizer_tp.zero_grad()
                
        batch_size , sequence_length , feature_dim = train_outputs[input_index].shape
            
        output = swe_model_tp(batch_input)
                    
        loss = mean_squared_error_tp(output, torch.from_numpy(train_outputs[input_index]).to(dtype=torch.float32))
      
        loss_in_batch = loss.item() * batch_input.shape[0]
        
        total_loss+=loss_in_batch
        
        loss.backward()
        optimizer_tp.step()
        
        if (input_index + 1) % 1000 == 0:
            print("Obtained Output:")
            print(output )
        
            
        if  (input_index + 1) % 100 == 0:
            print('Type: {} , Epoch Number: {} => Batch number :{} , loss value : {} '.format(data_type,epoch_number,input_index+1, loss.item()))
            print("=================================================")
        
            
    print('Type:{}, Epoch Number: {} => Avg loss value : {} '.format(data_type,epoch_number, total_loss / (total_batches * 1 )))
    
    return total_loss / (total_batches * 1 )



In [ ]:
swe_model_tp.train(True)

train_epoch_avg_losses_tp = []

start_time = time.time()

for index in range(12):
        
    temp_holder_tp = list(zip(tp_training_input, tp_training_output))
    random.shuffle(temp_holder_tp)
    
    train_input_batches_tp, train_output_batches_tp = zip(*temp_holder_tp)
    
    epoch_loss_tp = TrainModelTP(train_input_batches_tp, train_output_batches_tp, epoch_number = index+ 1)
    
    train_epoch_avg_losses_tp.append(epoch_loss_tp)
    
    scheduler_tp.step()

end_time = time.time()

print("Time Elapsed", end_time - start_time)

Type: Temporal , Epoch Number: 1 => Batch number :100 , loss value : 27.456493377685547 
Type: Temporal , Epoch Number: 1 => Batch number :200 , loss value : 79.79981231689453 
Type: Temporal , Epoch Number: 1 => Batch number :300 , loss value : 123.74552917480469 
Type: Temporal , Epoch Number: 1 => Batch number :400 , loss value : 8.13170337677002 
Type: Temporal , Epoch Number: 1 => Batch number :500 , loss value : 6.772975444793701 
Type: Temporal , Epoch Number: 1 => Batch number :600 , loss value : 21.380327224731445 
Type: Temporal , Epoch Number: 1 => Batch number :700 , loss value : 11.930166244506836 
Type: Temporal , Epoch Number: 1 => Batch number :800 , loss value : 13.460476875305176 
Type: Temporal , Epoch Number: 1 => Batch number :900 , loss value : 12.442276954650879 
Obtained Output:
tensor([[[-2.5502e-01],
         [-3.2738e-01],
         [-1.4974e-01],
         [-1.0465e-01],
         [-1.6250e-02],
         [ 2.0616e-01],
         [-2.2966e-01],
         [-6.5901e

Type: Temporal , Epoch Number: 1 => Batch number :1300 , loss value : 228.14010620117188 
Type: Temporal , Epoch Number: 1 => Batch number :1400 , loss value : 4.634644985198975 
Type: Temporal , Epoch Number: 1 => Batch number :1500 , loss value : 222.50607299804688 
Type: Temporal , Epoch Number: 1 => Batch number :1600 , loss value : 39.892852783203125 
Type: Temporal , Epoch Number: 1 => Batch number :1700 , loss value : 33.80820846557617 
Type: Temporal , Epoch Number: 1 => Batch number :1800 , loss value : 14.229633331298828 
Type: Temporal , Epoch Number: 1 => Batch number :1900 , loss value : 95.70640563964844 
Obtained Output:
tensor([[[ 1.0846e-01],
         [ 1.9377e-01],
         [ 1.2590e-01],
         [ 2.3879e-01],
         [ 3.1558e-01],
         [ 3.2880e-02],
         [ 3.9989e-02],
         [ 1.0060e-01],
         [-5.9763e-02],
         [-8.3315e-02],
         [-1.4168e-01],
         [-4.8318e-02],
         [ 1.7158e-01],
         [-3.4462e-02],
         [ 4.4425e-0

Type: Temporal , Epoch Number: 1 => Batch number :2500 , loss value : 87.86090850830078 
Type: Temporal , Epoch Number: 1 => Batch number :2600 , loss value : 78.84381866455078 
Type: Temporal , Epoch Number: 1 => Batch number :2700 , loss value : 32.73720932006836 
Type: Temporal , Epoch Number: 1 => Batch number :2800 , loss value : 31.376522064208984 
Type: Temporal , Epoch Number: 1 => Batch number :2900 , loss value : 19.023345947265625 
Obtained Output:
tensor([[[ 1.1717e-01],
         [-4.8602e-02],
         [-3.2046e-02],
         [-2.8751e-01],
         [-2.1556e-01],
         [-1.7374e-01],
         [-1.3079e-01],
         [-1.5425e-01],
         [-2.9257e-01],
         [-7.0418e-02],
         [-2.0975e-01],
         [-2.0933e-01],
         [-1.0774e-01],
         [-1.7606e-01],
         [-1.7203e-01],
         [ 6.2735e-02],
         [-1.6134e-01],
         [-3.6516e-02],
         [-1.6957e-01],
         [-2.8398e-02],
         [-1.4618e-01],
         [-1.9347e-01],
        

Type: Temporal , Epoch Number: 1 => Batch number :3700 , loss value : 9.508319854736328 
Type: Temporal , Epoch Number: 1 => Batch number :3800 , loss value : 12.510932922363281 
Type: Temporal , Epoch Number: 1 => Batch number :3900 , loss value : 115.62406158447266 
Obtained Output:
tensor([[[-3.1974e-03],
         [-3.9411e-02],
         [-3.4594e-02],
         [-1.2205e-01],
         [-2.7714e-02],
         [-7.4984e-02],
         [ 1.3449e-02],
         [ 2.7102e-02],
         [-3.5031e-02],
         [-1.0507e-01],
         [-4.7844e-02],
         [-1.2550e-01],
         [-2.3205e-01],
         [-1.9139e-01],
         [-1.8740e-01],
         [-2.0174e-01],
         [-1.4625e-01],
         [-2.0560e-01],
         [-2.5238e-01],
         [-6.9730e-02],
         [-1.6796e-01],
         [-2.3995e-01],
         [-1.0886e-01],
         [-1.1365e-01],
         [-7.7462e-03],
         [ 5.4742e-02],
         [ 3.7121e-02],
         [ 1.1323e-01],
         [ 4.1230e-02],
         [ 3.0277e

Type: Temporal , Epoch Number: 2 => Batch number :800 , loss value : 652.7606201171875 
Type: Temporal , Epoch Number: 2 => Batch number :900 , loss value : 21.093658447265625 
Obtained Output:
tensor([[[-1.8588e-01],
         [-2.9961e-01],
         [ 2.1898e-01],
         [ 2.3303e-01],
         [ 1.0939e-01],
         [ 2.4974e-01],
         [ 1.8200e-01],
         [ 2.3825e-01],
         [ 1.9595e-01],
         [-1.3765e-03],
         [ 1.4391e-01],
         [ 1.1017e-02],
         [ 2.0244e-01],
         [ 6.1487e-02],
         [ 1.9198e-01],
         [ 1.0663e-01],
         [ 1.5086e-01],
         [ 9.0518e-02],
         [ 1.6222e-01],
         [ 2.7659e-01],
         [ 4.7677e-01],
         [ 2.7117e-01],
         [ 4.1515e-01],
         [ 4.8633e-01],
         [ 3.0631e-01],
         [ 5.2068e-01],
         [ 5.1850e-01],
         [ 4.8220e-01],
         [ 6.9154e-01],
         [ 6.3542e-01],
         [ 5.2096e-01],
         [ 3.6308e-01],
         [ 4.6553e-01],
         [ 3.6

Obtained Output:
tensor([[[ 0.6252],
         [ 0.4335],
         [ 0.4699],
         [ 0.7061],
         [ 0.9025],
         [ 0.6820],
         [ 1.0093],
         [ 0.6313],
         [ 0.9634],
         [ 0.4060],
         [ 0.2514],
         [ 0.6134],
         [ 0.6460],
         [ 0.9390],
         [ 0.8572],
         [ 0.5392],
         [ 0.7157],
         [ 0.5734],
         [ 0.5752],
         [ 0.6689],
         [ 0.8786],
         [ 1.2444],
         [ 0.9782],
         [ 0.6030],
         [ 0.5316],
         [ 0.6593],
         [ 0.5307],
         [ 0.7480],
         [ 0.4628],
         [ 0.6092],
         [ 0.6157],
         [ 0.6960],
         [ 0.5466],
         [ 1.0924],
         [ 0.6520],
         [ 0.8046],
         [ 1.0228],
         [ 1.3346],
         [ 1.2864],
         [ 1.2031],
         [ 0.6315],
         [ 0.9127],
         [ 1.3340],
         [ 1.3191],
         [ 1.5500],
         [ 1.4301],
         [ 1.2259],
         [ 1.3608],
         [ 1.5960],
   

Type: Temporal , Epoch Number: 2 => Batch number :3100 , loss value : 14.699055671691895 
Type: Temporal , Epoch Number: 2 => Batch number :3200 , loss value : 3.7055130004882812 
Type: Temporal , Epoch Number: 2 => Batch number :3300 , loss value : 26.83802604675293 
Type: Temporal , Epoch Number: 2 => Batch number :3400 , loss value : 66.43262481689453 
Type: Temporal , Epoch Number: 2 => Batch number :3500 , loss value : 3.4975152015686035 
Type: Temporal , Epoch Number: 2 => Batch number :3600 , loss value : 10.501510620117188 
Type: Temporal , Epoch Number: 2 => Batch number :3700 , loss value : 15.74478816986084 
Type: Temporal , Epoch Number: 2 => Batch number :3800 , loss value : 11.43637466430664 
Type: Temporal , Epoch Number: 2 => Batch number :3900 , loss value : 10.459854125976562 


In [10]:
swe_model_tp.eval()

def TestModelTP(test_inputs, test_outputs):
    loss_value = 0
    losses = []
    
    print("========================")
    print('Testing the model.')
    print("========================")
    
    outputs = []
    outputs_loss = []
    nse_values = []
    
    mse_tp = nn.MSELoss()
    
    with torch.no_grad():
        for input_index , batch_input in enumerate(test_inputs):
            
            batch_size, sequence_length , output_feature_dim = test_outputs[input_index].shape
            
            #SWE_Model.transformers_1.apply(lambda module: setattr(module, "training", True))
            
            output = swe_model_tp(batch_input)
                        
            loss = mse_tp(output[0], torch.from_numpy(test_outputs[input_index]).to(dtype=torch.float32))
            
            outputs.append(output[0].detach().numpy())
            outputs_loss.append(loss.item())
            
            
            nse = he.evaluator(he.nse, output[0].reshape(sequence_length,1), np.array(test_outputs[input_index].reshape(sequence_length, output_feature_dim)))
            
            nse_values.append(nse)
            
            print('Location :{} , loss value : {}, nse:{} '.format(snotel_locations_info['Station Name'].values[input_index], loss.item(),nse))
            
            losses.append(loss.item())
    
    return (np.array(outputs), np.array(outputs_loss))
            



In [11]:
test_outputs_1_tp, test_losses_1_tp = TestModelTP(tp_testing_input_1,tp_testing_output_1)

Testing the model.


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 270, 1])) that is different to the input size (torch.Size([270, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Location :Adin Mtn , loss value : 14.48353385925293, nse:[-26.96605346] 
Location :Albro Lake , loss value : 8.270821571350098, nse:[0.73673568] 
Location :Aneroid Lake #2 , loss value : 13.05366039276123, nse:[0.58785154] 
Location :Baker Butte , loss value : 1.091412901878357, nse:[-2.85509899] 
Location :Baldy , loss value : 1.552376627922058, nse:[-0.11824293] 
Location :Banfield Mountain , loss value : 75.95669555664062, nse:[-4.56245723] 
Location :Banner Summit , loss value : 16.519390106201172, nse:[0.6313163] 
Location :Barker Lakes , loss value : 1.6991404294967651, nse:[0.9215624] 
Location :Base Camp , loss value : 2.2924001216888428, nse:[0.93406674] 
Location :Basin Creek , loss value : 0.7414177656173706, nse:[0.90624672] 
Location :Beagle Springs , loss value : 2.2341060638427734, nse:[0.6661961] 
Location :Bear Basin , loss value : 1.2684837579727173, nse:[0.91099267] 
Location :Bear Canyon , loss value : 11.88055419921875, nse:[-0.00236298] 
Location :Bear Creek , los

Location :Fourmile Lake , loss value : 240.56251525878906, nse:[-51.35882859] 
Location :Franklin Basin , loss value : 14.114784240722656, nse:[0.76169046] 
Location :Frohner Meadow , loss value : 6.631465911865234, nse:[0.0620617] 
Location :Galena , loss value : 4.01458740234375, nse:[0.84341652] 
Location :Galena Summit , loss value : 35.04035949707031, nse:[0.21037527] 
Location :Gallegos Peak , loss value : 1.6155513525009155, nse:[0.8351403] 
Location :Garver Creek , loss value : 1.8044592142105103, nse:[0.69845169] 
Location :Gerber Reservoir , loss value : 4.512955665588379, nse:[-5632.46696461] 
Location :Giveout , loss value : 2.940526008605957, nse:[0.63639575] 
Location :Gold Center , loss value : 12.42875862121582, nse:[-1.39549198] 
Location :Granite Creek , loss value : 19.074188232421875, nse:[0.50919195] 
Location :Granite Peak , loss value : 14.461390495300293, nse:[0.08753482] 
Location :Grassy Lake , loss value : 18.757226943969727, nse:[0.81656761] 
Location :Grave

Location :Owl Creek , loss value : 3.3969621658325195, nse:[-0.32675678] 
Location :Park Cone , loss value : 1.360627293586731, nse:[0.8362565] 
Location :Park Reservoir , loss value : 31.736621856689453, nse:[0.04651496] 
Location :Parrish Creek , loss value : 20.451194763183594, nse:[0.2438569] 
Location :Peterson Meadows , loss value : 2.431516408920288, nse:[0.81576928] 
Location :Phantom Valley , loss value : 0.4683283567428589, nse:[0.94943194] 
Location :Phillips Bench , loss value : 13.862648963928223, nse:[0.78937861] 
Location :Pickfoot Creek , loss value : 4.754886150360107, nse:[0.60337098] 
Location :Pine Creek Pass , loss value : 4.43891716003418, nse:[0.8377119] 
Location :Placer Basin , loss value : 2.1347856521606445, nse:[0.92849225] 
Location :Poison Flat , loss value : 4.184347152709961, nse:[-1.26710956] 
Location :Poorman Creek , loss value : 68.46406555175781, nse:[-0.73435481] 
Location :Pope Ridge , loss value : 17.618335723876953, nse:[-0.03239984] 
Location :

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/hydroeval/objective_functions.py:39: RuntimeWarning: divide by zero encountered in divide
  np.sum((evaluation - simulations) ** 2, axis=0, dtype=np.float64)


Location :Sevenmile Marsh , loss value : 240.96658325195312, nse:[-327.81220131] 
Location :Seventysix Creek , loss value : 2.682671546936035, nse:[0.65345049] 
Location :Shanghi Summit , loss value : 11.210620880126953, nse:[0.19579686] 
Location :Shell Creek , loss value : 1.1580325365066528, nse:[0.94197031] 
Location :Short Creek , loss value : 3.1748299598693848, nse:[-0.88314277] 
Location :Shower Falls , loss value : 13.054996490478516, nse:[0.76183505] 
Location :Signal Peak , loss value : 1.9915027618408203, nse:[-1.53324285] 
Location :Silvies , loss value : 11.693775177001953, nse:[-2.16148776] 
Location :Skalkaho Summit , loss value : 6.283570289611816, nse:[0.87903318] 
Location :Sleeping Woman , loss value : 24.930850982666016, nse:[-0.16340604] 
Location :Slug Creek Divide , loss value : 11.95737075805664, nse:[0.62967605] 
Location :Slumgullion , loss value : 5.009395599365234, nse:[0.74988423] 
Location :Snider Basin , loss value : 2.313772201538086, nse:[0.90537096] 


In [13]:
tp_obtained_1 = test_outputs_1_tp
actual_output_1 = tp_testing_output_1.reshape(323,270,1)

output_1_mse = nn.MSELoss()

nse_tp_1 = []

mse_tp_1 = []

output_tp_df_1 = pd.DataFrame()


for index in range(len(snotel_locations_info['Station Name'])):
    tp_output = tp_obtained_1[index,:,:]
        
    target_output = actual_output_1[index,:,:]
    
    nse_tp = he.evaluator(he.nse, tp_output.reshape(-1,), target_output.reshape(-1,))
    
    loss_tp = output_1_mse(torch.from_numpy(tp_output),torch.from_numpy(target_output))
    
    output_tp_df_1[snotel_locations_info['Station Name'].values[index]] = tp_output.reshape(-1,)
    
    nse_tp_1.append(nse_tp)
    
    mse_tp_1.append(loss_tp.item())

    

/var/folders/8g/08ng8zb12rqbw2s5thwqghgh0000gn/T/ipykernel_5635/739007839.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_tp_df_1[snotel_locations_info['Station Name'].values[index]] = tp_output.reshape(-1,)
/var/folders/8g/08ng8zb12rqbw2s5thwqghgh0000gn/T/ipykernel_5635/739007839.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_tp_df_1[snotel_locations_info['Station Name'].values[index]] = tp_output.reshape(-1,)
/var/folders/8g/08ng8zb12rqbw2s5thwqghgh0000gn/T/ipykernel_5635/739007839.py:22: PerformanceWa

In [14]:
test_outputs_2_tp, test_losses_2_tp = TestModelTP(tp_testing_input_2,tp_testing_output_2)

Testing the model.
Location :Adin Mtn , loss value : 4.1454548835754395, nse:[0.44054468] 
Location :Albro Lake , loss value : 7.059446334838867, nse:[0.77708193] 


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 270, 1])) that is different to the input size (torch.Size([270, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Location :Aneroid Lake #2 , loss value : 111.34507751464844, nse:[-2.26478589] 
Location :Baker Butte , loss value : 0.6652805805206299, nse:[0.73689444] 
Location :Baldy , loss value : 2.113438367843628, nse:[0.36044313] 
Location :Banfield Mountain , loss value : 89.51172637939453, nse:[-1.3932258] 
Location :Banner Summit , loss value : 4.648040771484375, nse:[0.92472425] 
Location :Barker Lakes , loss value : 2.241391897201538, nse:[0.90363025] 
Location :Base Camp , loss value : 17.630891799926758, nse:[0.30611436] 
Location :Basin Creek , loss value : 1.3590997457504272, nse:[0.77243308] 
Location :Beagle Springs , loss value : 8.472989082336426, nse:[-0.26945364] 
Location :Bear Basin , loss value : 3.3089599609375, nse:[0.89143716] 
Location :Bear Canyon , loss value : 3.710512161254883, nse:[0.66389168] 
Location :Bear Creek , loss value : 7.836259365081787, nse:[0.72995821] 
Location :Bear Mountain , loss value : 51.37846755981445, nse:[0.87555878] 
Location :Bear Saddle , lo

Location :Galena , loss value : 4.813776969909668, nse:[0.76715037] 
Location :Galena Summit , loss value : 72.4698257446289, nse:[-1.24990179] 
Location :Gallegos Peak , loss value : 7.347742080688477, nse:[0.48970004] 
Location :Garver Creek , loss value : 16.214597702026367, nse:[-0.03021779] 
Location :Gerber Reservoir , loss value : 6.528846740722656, nse:[-30.8288856] 
Location :Giveout , loss value : 2.0730783939361572, nse:[0.62444981] 
Location :Gold Center , loss value : 17.086896896362305, nse:[-0.80665476] 
Location :Granite Creek , loss value : 1.5970677137374878, nse:[0.91155459] 
Location :Granite Peak , loss value : 17.726043701171875, nse:[0.25909856] 
Location :Grassy Lake , loss value : 4.786078929901123, nse:[0.94901671] 
Location :Grave Creek , loss value : 17.429777145385742, nse:[0.25960975] 
Location :Grave Springs , loss value : 0.9062190055847168, nse:[0.86363283] 
Location :Green Lake , loss value : 58.534427642822266, nse:[0.3740782] 
Location :Green Mountai

Location :Phillips Bench , loss value : 7.17864465713501, nse:[0.84429832] 
Location :Pickfoot Creek , loss value : 6.675940990447998, nse:[0.10032547] 
Location :Pine Creek Pass , loss value : 1.4824168682098389, nse:[0.93684755] 
Location :Placer Basin , loss value : 5.142557621002197, nse:[0.79870921] 
Location :Poison Flat , loss value : 2.7992467880249023, nse:[0.79305372] 
Location :Poorman Creek , loss value : 104.08614349365234, nse:[0.31569392] 
Location :Pope Ridge , loss value : 8.020834922790527, nse:[0.86166655] 
Location :Porcupine , loss value : 16.81627082824707, nse:[-7.02404698] 
Location :Powder River Pass , loss value : 14.49669075012207, nse:[-0.29929124] 
Location :Prairie , loss value : 6.96734094619751, nse:[-12.53150269] 
Location :Quartz Peak , loss value : 45.508358001708984, nse:[0.12052161] 
Location :Railroad Overpass , loss value : 6.761570453643799, nse:[-105.22121592] 
Location :Rainy Pass , loss value : 28.4865665435791, nse:[0.86512374] 
Location :Red

In [15]:
tp_obtained_2 = test_outputs_2_tp
actual_output_2 = tp_testing_output_2.reshape(323,270,1)

output_2_mse = nn.MSELoss()

nse_tp_2 = []

mse_tp_2 = []

output_tp_df_2 = pd.DataFrame()


for index in range(len(snotel_locations_info['Station Name'])):
    tp_output = tp_obtained_2[index,:,:]
        
    target_output = actual_output_2[index,:,:]
    
    nse_tp = he.evaluator(he.nse, tp_output.reshape(-1,), target_output.reshape(-1,))
    
    loss_tp = output_2_mse(torch.from_numpy(tp_output),torch.from_numpy(target_output))
    
    output_tp_df_2[snotel_locations_info['Station Name'].values[index]] = tp_output.reshape(-1,)
    
    nse_tp_2.append(nse_tp)
    
    mse_tp_2.append(loss_tp.item())

    

/var/folders/8g/08ng8zb12rqbw2s5thwqghgh0000gn/T/ipykernel_5635/752484910.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_tp_df_2[snotel_locations_info['Station Name'].values[index]] = tp_output.reshape(-1,)
/var/folders/8g/08ng8zb12rqbw2s5thwqghgh0000gn/T/ipykernel_5635/752484910.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_tp_df_2[snotel_locations_info['Station Name'].values[index]] = tp_output.reshape(-1,)
/var/folders/8g/08ng8zb12rqbw2s5thwqghgh0000gn/T/ipykernel_5635/752484910.py:22: PerformanceWa

In [16]:
test_outputs_3_tp, test_losses_3_tp = TestModelTP(tp_testing_input_3,tp_testing_output_3)

Testing the model.
Location :Adin Mtn , loss value : 6.820126056671143, nse:[-0.80698124] 
Location :Albro Lake , loss value : 68.39990997314453, nse:[-0.17998597] 


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 270, 1])) that is different to the input size (torch.Size([270, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Location :Aneroid Lake #2 , loss value : 4.3923797607421875, nse:[0.91980663] 
Location :Baker Butte , loss value : 1.6723281145095825, nse:[-5.45209827] 
Location :Baldy , loss value : 14.172006607055664, nse:[-41.89874046] 
Location :Banfield Mountain , loss value : 19.47812843322754, nse:[0.64210383] 
Location :Banner Summit , loss value : 23.770580291748047, nse:[0.69877842] 
Location :Barker Lakes , loss value : 47.540653228759766, nse:[-0.35381327] 
Location :Base Camp , loss value : 15.974681854248047, nse:[0.63348125] 
Location :Basin Creek , loss value : 5.358827590942383, nse:[0.63388378] 
Location :Beagle Springs , loss value : 7.107316970825195, nse:[0.58178073] 
Location :Bear Basin , loss value : 1.7506171464920044, nse:[0.94013135] 
Location :Bear Canyon , loss value : 10.183842658996582, nse:[0.4914345] 
Location :Bear Creek , loss value : 17.4053955078125, nse:[0.22414828] 
Location :Bear Mountain , loss value : 48.243858337402344, nse:[0.9018261] 
Location :Bear Saddl

Location :Galena , loss value : 7.327504634857178, nse:[0.73409483] 
Location :Galena Summit , loss value : 35.67348861694336, nse:[0.35072061] 
Location :Gallegos Peak , loss value : 3.262831449508667, nse:[-0.87011643] 
Location :Garver Creek , loss value : 3.8251407146453857, nse:[0.85930458] 
Location :Gerber Reservoir , loss value : 1.8590608835220337, nse:[-49.05301149] 
Location :Giveout , loss value : 11.35464859008789, nse:[-2.4073813] 
Location :Gold Center , loss value : 28.325735092163086, nse:[-3.93406333] 
Location :Granite Creek , loss value : 4.524191379547119, nse:[0.89202646] 
Location :Granite Peak , loss value : 70.90846252441406, nse:[-2.03447013] 
Location :Grassy Lake , loss value : 8.35258960723877, nse:[0.93925658] 
Location :Grave Creek , loss value : 5.164220809936523, nse:[0.93318114] 
Location :Grave Springs , loss value : 0.6077705025672913, nse:[0.9153375] 
Location :Green Lake , loss value : 71.13865661621094, nse:[-0.13318898] 
Location :Green Mountain 

Location :Phillips Bench , loss value : 7.539137840270996, nse:[0.9292301] 
Location :Pickfoot Creek , loss value : 4.689852714538574, nse:[0.82524371] 
Location :Pine Creek Pass , loss value : 5.028178691864014, nse:[0.81098939] 
Location :Placer Basin , loss value : 45.83066940307617, nse:[0.49086479] 
Location :Poison Flat , loss value : 74.88480377197266, nse:[-2.76813901] 
Location :Poorman Creek , loss value : 41.72914123535156, nse:[0.86835479] 
Location :Pope Ridge , loss value : 5.543013095855713, nse:[0.85127072] 
Location :Porcupine , loss value : 10.151972770690918, nse:[0.48993268] 
Location :Powder River Pass , loss value : 10.68641185760498, nse:[0.66853842] 
Location :Prairie , loss value : 0.8575648665428162, nse:[0.25355917] 
Location :Quartz Peak , loss value : 13.648571014404297, nse:[0.81571409] 
Location :Railroad Overpass , loss value : 1.265572190284729, nse:[-12.71024417] 
Location :Rainy Pass , loss value : 72.00606536865234, nse:[0.73391309] 
Location :Red Hi

In [17]:
tp_obtained_3 = test_outputs_1_tp
actual_output_3 = tp_testing_output_3.reshape(323,270,1)

output_3_mse = nn.MSELoss()

nse_tp_3 = []

mse_tp_3 = []

output_tp_df_3 = pd.DataFrame()


for index in range(len(snotel_locations_info['Station Name'])):
    tp_output = tp_obtained_3[index,:,:]
        
    target_output = actual_output_3[index,:,:]
    
    nse_tp = he.evaluator(he.nse, tp_output.reshape(-1,), target_output.reshape(-1,))
    
    loss_tp = output_3_mse(torch.from_numpy(tp_output),torch.from_numpy(target_output))
    
    output_tp_df_3[snotel_locations_info['Station Name'].values[index]] = tp_output.reshape(-1,)
    
    nse_tp_3.append(nse_tp)
    
    mse_tp_3.append(loss_tp.item())

    

/var/folders/8g/08ng8zb12rqbw2s5thwqghgh0000gn/T/ipykernel_5635/2339497515.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_tp_df_3[snotel_locations_info['Station Name'].values[index]] = tp_output.reshape(-1,)
/var/folders/8g/08ng8zb12rqbw2s5thwqghgh0000gn/T/ipykernel_5635/2339497515.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_tp_df_3[snotel_locations_info['Station Name'].values[index]] = tp_output.reshape(-1,)
/var/folders/8g/08ng8zb12rqbw2s5thwqghgh0000gn/T/ipykernel_5635/2339497515.py:22: Performanc

In [18]:
test_outputs_4_tp, test_losses_4_tp = TestModelTP(tp_testing_input_4,tp_testing_output_4)

Testing the model.
Location :Adin Mtn , loss value : 27.759740829467773, nse:[0.55243889] 
Location :Albro Lake , loss value : 28.102643966674805, nse:[0.61382926] 


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 270, 1])) that is different to the input size (torch.Size([270, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Location :Aneroid Lake #2 , loss value : 60.76203155517578, nse:[0.55579551] 
Location :Baker Butte , loss value : 2.4832096099853516, nse:[0.29044888] 
Location :Baldy , loss value : 10.000076293945312, nse:[-0.49977009] 
Location :Banfield Mountain , loss value : 16.002113342285156, nse:[0.6029016] 
Location :Banner Summit , loss value : 8.428963661193848, nse:[0.96339541] 
Location :Barker Lakes , loss value : 11.5687255859375, nse:[0.60249907] 
Location :Base Camp , loss value : 6.898967742919922, nse:[0.92919755] 
Location :Basin Creek , loss value : 2.7935755252838135, nse:[0.45092985] 
Location :Beagle Springs , loss value : 6.063449382781982, nse:[0.69861878] 
Location :Bear Basin , loss value : 4.615304470062256, nse:[0.88870451] 
Location :Bear Canyon , loss value : 3.9008147716522217, nse:[0.95703546] 
Location :Bear Creek , loss value : 60.295555114746094, nse:[0.28807826] 
Location :Bear Mountain , loss value : 15.291017532348633, nse:[0.95429985] 
Location :Bear Saddle , 

Location :Galena , loss value : 3.631305694580078, nse:[0.9676277] 
Location :Galena Summit , loss value : 10.283318519592285, nse:[0.93944348] 
Location :Gallegos Peak , loss value : 3.0010986328125, nse:[0.77810921] 
Location :Garver Creek , loss value : 1.0849374532699585, nse:[0.93761093] 
Location :Gerber Reservoir , loss value : 1.5266021490097046, nse:[-0.96805399] 
Location :Giveout , loss value : 8.793240547180176, nse:[0.71564031] 
Location :Gold Center , loss value : 17.523874282836914, nse:[0.44539976] 
Location :Granite Creek , loss value : 74.07523345947266, nse:[0.39419814] 
Location :Granite Peak , loss value : 81.44174194335938, nse:[0.56558645] 
Location :Grassy Lake , loss value : 31.395540237426758, nse:[0.85170016] 
Location :Grave Creek , loss value : 1.5537495613098145, nse:[0.95214943] 
Location :Grave Springs , loss value : 3.4121313095092773, nse:[0.62937405] 
Location :Green Lake , loss value : 26.264320373535156, nse:[0.74032073] 
Location :Green Mountain , 

Location :Phillips Bench , loss value : 58.906063079833984, nse:[0.73047775] 
Location :Pickfoot Creek , loss value : 1.9631297588348389, nse:[0.87869351] 
Location :Pine Creek Pass , loss value : 11.860239028930664, nse:[0.67705562] 
Location :Placer Basin , loss value : 5.373500823974609, nse:[0.88678211] 
Location :Poison Flat , loss value : 16.885835647583008, nse:[0.90619142] 
Location :Poorman Creek , loss value : 6.52294921875, nse:[0.96439292] 
Location :Pope Ridge , loss value : 3.3800010681152344, nse:[0.91404146] 
Location :Porcupine , loss value : 1.7184114456176758, nse:[0.17183955] 
Location :Powder River Pass , loss value : 3.107775926589966, nse:[0.74337749] 
Location :Prairie , loss value : 7.9189229011535645, nse:[0.30343723] 
Location :Quartz Peak , loss value : 11.626190185546875, nse:[0.77849993] 
Location :Railroad Overpass , loss value : 1.4875695705413818, nse:[-3.6217321] 
Location :Rainy Pass , loss value : 21.848337173461914, nse:[0.88631962] 
Location :Red H

In [19]:
tp_obtained_4 = test_outputs_4_tp
actual_output_4 = tp_testing_output_4.reshape(323,270,1)

output_4_mse = nn.MSELoss()

nse_tp_4 = []

mse_tp_4 = []

output_tp_df_4 = pd.DataFrame()


for index in range(len(snotel_locations_info['Station Name'])):
    tp_output = tp_obtained_4[index,:,:]
        
    target_output = actual_output_4[index,:,:]
    
    nse_tp = he.evaluator(he.nse, tp_output.reshape(-1,), target_output.reshape(-1,))
    
    loss_tp = output_4_mse(torch.from_numpy(tp_output),torch.from_numpy(target_output))
    
    output_tp_df_4[snotel_locations_info['Station Name'].values[index]] = tp_output.reshape(-1,)
    
    nse_tp_4.append(nse_tp)
    
    mse_tp_4.append(loss_tp.item())

    

/var/folders/8g/08ng8zb12rqbw2s5thwqghgh0000gn/T/ipykernel_5635/993336581.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_tp_df_4[snotel_locations_info['Station Name'].values[index]] = tp_output.reshape(-1,)
/var/folders/8g/08ng8zb12rqbw2s5thwqghgh0000gn/T/ipykernel_5635/993336581.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_tp_df_4[snotel_locations_info['Station Name'].values[index]] = tp_output.reshape(-1,)
/var/folders/8g/08ng8zb12rqbw2s5thwqghgh0000gn/T/ipykernel_5635/993336581.py:22: PerformanceWa

In [20]:
test_outputs_5_tp, test_losses_5_tp = TestModelTP(tp_testing_input_5,tp_testing_output_5)

Testing the model.
Location :Adin Mtn , loss value : 10.257227897644043, nse:[0.65138971] 
Location :Albro Lake , loss value : 52.47861099243164, nse:[-0.10990212] 


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 270, 1])) that is different to the input size (torch.Size([270, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Location :Aneroid Lake #2 , loss value : 41.00379943847656, nse:[0.6508521] 
Location :Baker Butte , loss value : 1.5985910892486572, nse:[0.84758267] 
Location :Baldy , loss value : 2.8523988723754883, nse:[0.83875828] 
Location :Banfield Mountain , loss value : 2.326141595840454, nse:[0.96371652] 
Location :Banner Summit , loss value : 16.559541702270508, nse:[0.85198232] 
Location :Barker Lakes , loss value : 39.229248046875, nse:[-0.25305602] 
Location :Base Camp , loss value : 17.39763069152832, nse:[0.67587017] 
Location :Basin Creek , loss value : 3.306095600128174, nse:[0.55445678] 
Location :Beagle Springs , loss value : 3.641087770462036, nse:[0.77287373] 
Location :Bear Basin , loss value : 108.50740814208984, nse:[-0.38946415] 
Location :Bear Canyon , loss value : 3.9407100677490234, nse:[0.88183949] 
Location :Bear Creek , loss value : 30.04434585571289, nse:[0.50502764] 
Location :Bear Mountain , loss value : 382.9693298339844, nse:[0.31202138] 
Location :Bear Saddle , lo

Location :Galena , loss value : 9.449484825134277, nse:[0.7899137] 
Location :Galena Summit , loss value : 31.460695266723633, nse:[0.54008263] 
Location :Gallegos Peak , loss value : 2.4025990962982178, nse:[0.87333246] 
Location :Garver Creek , loss value : 12.15263843536377, nse:[0.52637769] 
Location :Gerber Reservoir , loss value : 0.5117056369781494, nse:[0.84819516] 
Location :Giveout , loss value : 9.524759292602539, nse:[0.40843412] 
Location :Gold Center , loss value : 14.51329231262207, nse:[0.55931711] 
Location :Granite Creek , loss value : 55.242000579833984, nse:[-0.13474093] 
Location :Granite Peak , loss value : 21.779613494873047, nse:[0.6597445] 
Location :Grassy Lake , loss value : 103.47647094726562, nse:[0.41199429] 
Location :Grave Creek , loss value : 31.584142684936523, nse:[0.4426398] 
Location :Grave Springs , loss value : 5.254258155822754, nse:[0.65938055] 
Location :Green Lake , loss value : 23.40330696105957, nse:[0.8144172] 
Location :Green Mountain , lo

Location :Phillips Bench , loss value : 57.52500534057617, nse:[0.62143196] 
Location :Pickfoot Creek , loss value : 3.680598258972168, nse:[0.78597153] 
Location :Pine Creek Pass , loss value : 38.49972152709961, nse:[0.24009718] 
Location :Placer Basin , loss value : 46.02742385864258, nse:[0.02893701] 
Location :Poison Flat , loss value : 43.42416000366211, nse:[0.32053168] 
Location :Poorman Creek , loss value : 215.2664794921875, nse:[0.31552519] 
Location :Pope Ridge , loss value : 9.086121559143066, nse:[0.83799991] 
Location :Porcupine , loss value : 2.1067025661468506, nse:[0.69294571] 
Location :Powder River Pass , loss value : 0.5918039083480835, nse:[0.97016768] 
Location :Prairie , loss value : 8.970154762268066, nse:[0.17133506] 
Location :Quartz Peak , loss value : 104.71435546875, nse:[0.40790369] 
Location :Railroad Overpass , loss value : 3.644124984741211, nse:[-0.27321814] 
Location :Rainy Pass , loss value : 90.60652160644531, nse:[0.44834605] 
Location :Red Hill ,

In [21]:
tp_obtained_5 = test_outputs_5_tp
actual_output_5 = tp_testing_output_5.reshape(323,270,1)

output_5_mse = nn.MSELoss()

nse_tp_5 = []

mse_tp_5 = []

output_tp_df_5 = pd.DataFrame()


for index in range(len(snotel_locations_info['Station Name'])):
    tp_output = tp_obtained_5[index,:,:]
        
    target_output = actual_output_5[index,:,:]
    
    nse_tp = he.evaluator(he.nse, tp_output.reshape(-1,), target_output.reshape(-1,))
    
    loss_tp = output_5_mse(torch.from_numpy(tp_output),torch.from_numpy(target_output))
    
    output_tp_df_5[snotel_locations_info['Station Name'].values[index]] = tp_output.reshape(-1,)
    
    nse_tp_5.append(nse_tp)
    
    mse_tp_5.append(loss_tp.item())

    

/var/folders/8g/08ng8zb12rqbw2s5thwqghgh0000gn/T/ipykernel_5635/3976043456.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_tp_df_5[snotel_locations_info['Station Name'].values[index]] = tp_output.reshape(-1,)
/var/folders/8g/08ng8zb12rqbw2s5thwqghgh0000gn/T/ipykernel_5635/3976043456.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_tp_df_5[snotel_locations_info['Station Name'].values[index]] = tp_output.reshape(-1,)
/var/folders/8g/08ng8zb12rqbw2s5thwqghgh0000gn/T/ipykernel_5635/3976043456.py:22: Performanc

In [22]:
# For NSE calculation

def getNSECategoryDict(nse_values):
    nse_dict = {"<0" : 0 , ">=0 & <0.3": 0 , ">=0.3 & <0.5": 0 , ">=0.5 & <0.75" : 0 , ">=0.75": 0}
    
    for nse in nse_values:
        if nse < 0:
            nse_dict['<0'] = nse_dict['<0'] + 1
    
        if nse >=0 and nse < 0.3:
            nse_dict[">=0 & <0.3"] = nse_dict[">=0 & <0.3"] + 1
    
        if nse>=0.3 and nse<0.5:
            nse_dict[">=0.3 & <0.5"] = nse_dict[">=0.3 & <0.5"] + 1
    
        if nse>=0.5 and nse<0.75:
            nse_dict[">=0.5 & <0.75"] = nse_dict[">=0.5 & <0.75"] + 1
    
        if nse>=0.75:
            nse_dict[">=0.75"] = nse_dict[">=0.75"] + 1
    
    return nse_dict
    


In [23]:
nse_tp_1_dict = getNSECategoryDict(nse_tp_1)

print("TP")
print(nse_tp_1_dict)


TP
{'<0': 122, '>=0 & <0.3': 23, '>=0.3 & <0.5': 18, '>=0.5 & <0.75': 61, '>=0.75': 99}


In [24]:
nse_tp_2_dict = getNSECategoryDict(nse_tp_2)

print("TP")
print(nse_tp_2_dict)


TP
{'<0': 86, '>=0 & <0.3': 24, '>=0.3 & <0.5': 30, '>=0.5 & <0.75': 51, '>=0.75': 132}


In [25]:
nse_tp_3_dict = getNSECategoryDict(nse_tp_3)

print("TP")
print(nse_tp_3_dict)


TP
{'<0': 71, '>=0 & <0.3': 26, '>=0.3 & <0.5': 31, '>=0.5 & <0.75': 93, '>=0.75': 102}


In [26]:
nse_tp_4_dict = getNSECategoryDict(nse_tp_4)

print("TP")
print(nse_tp_4_dict)


TP
{'<0': 23, '>=0 & <0.3': 30, '>=0.3 & <0.5': 35, '>=0.5 & <0.75': 97, '>=0.75': 138}


In [27]:
nse_tp_5_dict = getNSECategoryDict(nse_tp_5)

print("TP")
print(nse_tp_5_dict)


TP
{'<0': 47, '>=0 & <0.3': 42, '>=0.3 & <0.5': 55, '>=0.5 & <0.75': 87, '>=0.75': 92}


In [28]:
lstm_mse_df = pd.DataFrame()

lstm_mse_df[2014] = mse_tp_1
lstm_mse_df[2006] = mse_tp_2
lstm_mse_df[2017] = mse_tp_3
lstm_mse_df[2016] = mse_tp_4
lstm_mse_df[2007] = mse_tp_5

lstm_mse_df.to_csv('/Users/krishuthapa/Desktop/Results/Attention_T_363_MSE.csv')

In [30]:
output_tp_df_1.to_csv('/Users/krishuthapa/Desktop/Results/2014/Attention_T_2014_363_predict.csv')

In [31]:
output_tp_df_2.to_csv('/Users/krishuthapa/Desktop/Results/2006/Attention_T_2006_363_predict.csv')

In [32]:
output_tp_df_3.to_csv('/Users/krishuthapa/Desktop/Results/2017/Attention_T_2017_363_predict.csv')

In [33]:
output_tp_df_4.to_csv('/Users/krishuthapa/Desktop/Results/2016/Attention_T_2016_363_predict.csv')

In [34]:
output_tp_df_5.to_csv('/Users/krishuthapa/Desktop/Results/2007/Attention_T_2007_363_predict.csv')

In [93]:
output_tp_df_1.to_csv('/Users/krishuthapa/Desktop/Results/2014/Attention_T_Concat_2014_predict.csv')
output_tp_df_2.to_csv('/Users/krishuthapa/Desktop/Results/2006/Attention_T_Concat_2006_predict.csv')
output_tp_df_3.to_csv('/Users/krishuthapa/Desktop/Results/2017/Attention_T_Concat_2017_predict.csv')
output_tp_df_4.to_csv('/Users/krishuthapa/Desktop/Results/2016/Attention_T_Concat_2016_predict.csv')
output_tp_df_5.to_csv('/Users/krishuthapa/Desktop/Results/2007/Attention_T_Concat_2007_predict.csv')
